## Milestone 3 notebook is comprised of three sections:

### 1) Data joining with Pandas
### 2) Data enrichment with Spotipy from Spotify Web API
### 3) Data visualization with seaborn

--------------------------------------------------------------------------
***Note: We do not recommend running this notebook as all in all, it will take about 13-14 hours to complete. We ran it in its three separate components over a period of days to finalize.***

### Import necessary packages

In [ ]:
import time
import math
import pickle
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

from os import listdir
from os.path import isfile, join
sns.set()



from itertools import chain

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 

client_id = "9c828632c4884d67b66fdd35545b7ba8"
client_secret = "d846b33b9b154046ba46a27e3f8f684b"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## 1) Data joining with Pandas

### Spot check an individual songs file to explore its format and data.

In [ ]:
df = pd.read_csv('../data/Songs/songs285.csv')

In [ ]:
print(df.shape)
list(df.columns)

This is a lot of data: ball-park 65,000 rows in 1,000 files, totaling around 65,000,000 observations – or songs in playlists, many of which are certainly repeated.

In [ ]:
df.head()

In [ ]:
# Some very different playlist lengths
df.groupby('pid')['track_uri'].nunique()[10:20]

In [ ]:
all_files = listdir('../data/Songs')
print(len(all_files))
all_files[0:3]

### Structuring the data
A reasonable first step to slim down the size of the dataset without losing information or fidelity, is to parse through all the files to create a reference table/file of all songs and their metadata. Each playlist can then be stored as a simple named object, where the name is the overall playlist id and its value a vector of song ids.

### Looping over all our files to fill out the master DataFrame (songs) and Series (playlists)

In [ ]:
start_time = time.time()
loop_start = time.time()

# List of all files
all_files = listdir('../data/Songs')
# Ditch half the playlists; 500,000 playlists is plenty
all_files = all_files[0:200]

# Load first file to get columns (standard across all)
df = pd.read_csv('../data/Songs/' + all_files[0])

# Master DataFrame of all unique songs included across all playlists
#songs = pd.DataFrame(columns = list(df.columns)[2:])
songs = pd.DataFrame()

# Master Series of playlists and the songs included in each
playlists = pd.Series()

# Aggregator functions to limit to one row per song and count occurrences across playlists
a1 = dict()
for key in df.columns[2:]:
    a1[key] = 'first'
a1['track_uri'] = 'count'

# Aggregator to consolidate into sum of songs across playlists
a2= dict()
for key in df.columns[2:]:
    a2[key] = 'first'
del a2['track_uri']
a2['count'] = 'sum'

# Loop over each file to extract data
for i, file in enumerate(all_files):
    # split on "." to split into "filename" and "csv"
    # Then select "filename" and ditch the first five letters "songs"
    filenum = file.split(".")[0][5:]
    
    # Load file and store in temporary dataframe
    fdf = pd.read_csv('../data/Songs/' + file)
    
    # --- SONGS IN FILE ---
    fdf_counts = fdf.iloc[:, 2:]
    fdf_counts = fdf_counts.groupby('track_uri').agg(a1)
    fdf_counts.rename(columns = {'track_uri': 'count'}, inplace = True)
    
    # Add to df of unique songs, update counters, and remove duplicates
    songs = songs.append(fdf_counts)
    
    # -- SONGS IN EACH PLAYLIST --
    # Songs included in every playlist (ordered) in file
    # For each playlist, get list of track_uri's (unique identifiers)
    songs_in_playlist = fdf.groupby('pid')['track_uri'].unique()

    # Update index to be not the pid in file (id), but a combination of them
    #songs_in_playlist.index = [filenum + '_' + str(pid) for pid in songs_in_playlist.index.values]
    songs_in_playlist.index = list(map(lambda x: filenum + '_' + str(x), songs_in_playlist.index.values))
    
    # Add playlists to master Series of all playlists
    playlists = playlists.append(songs_in_playlist)
    
    # Every 50 files, consolidate the songs table so it doesn't grow too big
    if (i+1)%25 == 0: 
        print('{}/{} -- {} s'.format(i+1, len(all_files), time.time() - loop_start))
        loop_start = time.time()
        songs = songs.groupby('track_uri').agg(a2, sort = True)
        print('   Consolidation: {} s'.format(time.time() - loop_start))
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Do a final consolidation just to be safe (should be very fast)
# Add song ID to table, now that it only contains unique songs
start_time = time.time()
songs_counts = songs.groupby('track_uri').agg(a2)
songs_counts['song_id'] = np.arange(len(songs_counts))
print("--- %s seconds ---" % (time.time() - start_time))

print(songs_counts.shape)
display(songs_counts.head())

In [ ]:
# Replace playlist track_uri with song_id
start_time = time.time()
loop_start = time.time()

playlists_songids = pd.Series(index = playlists.index)
playlists_songids = playlists_songids.astype(object)

i = 0
for ind, row in playlists.items():
    songids = np.array(songs_counts.loc[row, 'song_id'], 'int')
    playlists_songids.loc[str(ind)] = songids
    
    i += 1
    if i % (len(playlists)/20) == 0 == 0: 
        print('{}/{} -- {} s'.format(i, int(len(playlists)), time.time() - loop_start))
        loop_start = time.time()
    
print("--- %s seconds ---" % (time.time() - start_time))

print(playlists_songids.shape)
print(playlists_songids.head())

In [ ]:
# Change songs table to have song id as index and track_uri as column
# We will be doing lookups on song_id while running
songs_counts_id = songs_counts.copy()
songs_counts_id['track_uri'] = songs_counts_id.index.values
songs_counts_id.set_index('song_id', inplace = True)
songs_counts_id.head()

### Write files to disk (csv and pickle)
Use `pd.read_pickle` to easily read back in a data frame or series with the exact same structure as the one you dumped.

In [ ]:
songs_counts_id.to_csv('../data/songs_counts_'+str(len(all_files))+'.csv')
songs_counts_id.to_pickle('../data/songs_counts_'+str(len(all_files))+'.pkl')

In [ ]:
playlists_songids.to_csv('../data/playlists_song_ids_'+str(len(all_files))+'.csv', header = False)
playlists_songids.to_pickle('../data/playlists_song_ids_'+str(len(all_files))+'.pkl')

## 2) Data enrichment with Spotipy from Spotify Web API


### Ingest saved raw data

This occurred asynchronously so was easier to export and ingest again.

In [ ]:
df = pd.read_pickle('../data/songs_counts_200.pkl')
df

### Enrich with new track features from Spotify

In [ ]:
# define batches
batch_size = 100
num_batches = math.ceil(len(df)/batch_size)

# initialize list to save API calls
track_features = []

start_time = time.time()

# looping through the batches
for i in range(num_batches):
    
    
    # define start and end of the batch
    start_point = i*batch_size
    end_point = min(start_point + batch_size, len(df))
    
    # API call
    track_list = list(df['track_uri'][start_point:end_point])
    track_features.extend(sp.audio_features(track_list))

    if i%100 == 0:
        print('{}/{}, {}s'.format(i, num_batches, time.time()-start_time))
        start_time = time.time()

In [ ]:
track_features_df = pd.DataFrame(track_features)
track_features_df

In [ ]:
track_features_df.to_csv('../data/track_features'+str(counter)+'.csv')
track_features_df.to_pickle('../data/track_features'+str(counter)+'.pkl')
counter += 1

### Enrich with new artist features from Spotify

In [ ]:
unique_artists = list(df['artist_uri'].unique())
len(unique_artists)

In [ ]:
# define batches
batch_size = 50
num_batches = math.ceil(len(unique_artists)/batch_size)

# initialize list to save API calls
artist_info = []

start_time = time.time()

# looping through the batches
for i in range(num_batches):
    
    
    # define start and end of the batch
    start_point = i*batch_size
    end_point = min(start_point + batch_size, len(df))
    
    # API call
    artist_list = unique_artists[start_point:end_point]
    artist_info.extend(sp.artists(artist_list)['artists'])

    if i%100 == 0:
        print('{}/{}, {}s'.format(i, num_batches, time.time()-start_time))
        start_time = time.time()

In [ ]:
artist_info_df = pd.DataFrame(artist_info)
artist_info_df

In [ ]:
len(set(list(chain.from_iterable(artist_info_df['genres']))))

In [ ]:
counter = 0

In [ ]:
artist_info_df.to_csv('../data/artist_info'+str(counter)+'.csv')
artist_info_df.to_pickle('../data/artist_info'+str(counter)+'.pkl')
counter += 1

### Enrich with new artist features from Spotify

In [ ]:
unique_albums = list(df['album_uri'].unique())
len(unique_albums)

In [ ]:
# albums_exist = pd.read_pickle('../data/album_info1.pkl')
albums_exist = list(albums_exist['uri'])
print(len(albums_exist))
# albums_exist

In [ ]:
len(set(unique_albums_new))

In [ ]:
# unique_albums_new = [i for i in unique_albums if i not in albums_exist]
unique_albums_new = list(set(unique_albums) - set(albums_exist))
print(len(unique_albums_new))
# unique_albums_new

In [ ]:
unique_albums = unique_albums_new[140000:]

In [ ]:
len(unique_albums)

In [ ]:
# define batches
batch_size = 20
num_batches = math.ceil(len(unique_albums)/batch_size)

# initialize list to save API calls
album_info = []

start_time = time.time()

# looping through the batches
for i in range(num_batches):
    
    
    # define start and end of the batch
    start_point = i*batch_size
    end_point = min(start_point + batch_size, len(df))
    
    # API call
    album_list = unique_albums[start_point:end_point]
    album_info.extend(sp.albums(album_list)['albums'])

    if i%100 == 0:
        print('{}/{}, {}s'.format(i, num_batches, time.time()-start_time))
        start_time = time.time()
    

In [ ]:
# Eliminates None values
album_info = [i for i in album_info if i is not None]

In [ ]:
album_info_df = pd.DataFrame(album_info)
album_info_df

In [ ]:
album_info_df.to_csv('../data/album_info'+str(counter)+'.csv')
album_info_df.to_pickle('../data/album_info'+str(counter)+'.pkl')
counter += 1

### Join all gathered enriched data together

In [ ]:
# Rename columns just for albums for clarity
album_columns = ['genres','popularity','release_date','uri']

albums1 = pd.read_csv('../data/album_info1.csv', usecols=album_columns)
albums2 = pd.read_csv('../data/album_info2.csv', usecols=album_columns)
albums3 = pd.read_csv('../data/album_info3.csv', usecols=album_columns)
albums4 = pd.read_csv('../data/album_info4.csv', usecols=album_columns)
albums5 = pd.read_csv('../data/album_info5.csv', usecols=album_columns)
albums6 = pd.read_csv('../data/album_info6.csv', usecols=album_columns)
albums = pd.concat([albums1, albums2, albums3, albums4, albums5, albums6], axis=0, ignore_index=True)

albums = albums.rename(columns={'genres': 'album_genres', 'popularity': 'album_popularity', 'release_date': 'album_release_date', 'uri': 'album_uri'})
albums = albums.drop_duplicates()

albums

In [ ]:
# Rename columns just for artists for clarity
artist_columns = ['genres','popularity','uri']
artists = pd.read_csv('../data/artist_info1.csv', usecols=artist_columns)

artists = artists.rename(columns={'genres': 'artist_genres', 'popularity': 'artist_popularity', 'uri': 'artist_uri'})
artists = artists.drop_duplicates()

artists

In [ ]:
# Rename columns just for tracks for clarity
track_columns = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature','uri']
tracks = pd.read_csv('../data/track_features3.csv', usecols=track_columns)

tracks = tracks.rename(columns={'uri': 'track_uri'})
tracks = tracks.drop_duplicates()

tracks

In [ ]:
master = pd.read_pickle('../data/songs_counts_200.pkl')
master['song_id'] = master.index
master

In [ ]:
master = master.merge(track_features, on='track_uri', suffixes=(None, '_tracks'))
master = master.merge(artists, on='artist_uri', suffixes=(None, '_artists'))
master = master.merge(albums, on='album_uri', suffixes=(None, '_albums'))
master = master.set_index('song_id')
master

### Save final enriched dataset to master

In [ ]:
master.to_csv('../data/master200.csv')
master.to_pickle('../data/master200.pkl')

## 3) Data visualization with seaborn

### Ingest saved raw data

This occurred asynchronously so was easier to export and ingest again.

In [ ]:
songs = pd.read_pickle('../../data/200_first_playlists/master200.pkl')
playlists = pd.read_pickle('../../data/200_first_playlists/playlists_song_ids_200.pkl')

In [ ]:
songs.columns

In [ ]:
# Distribution of song appearances across playlists
plt.subplots(figsize=(10,5))
plt.hist(songs.loc[(songs['count'] > 1) & (songs['count'] < 50), 'count'], bins = 30)
plt.suptitle('Distribution of song appearances across playlists')
plt.title('Filtered to 1 < frequency < 50 as there is an extremely long right tail');

In [ ]:
# Distribution of number of songs in playlist
plt.subplots(figsize=(10,5))
playlist_lengths = np.array([len(p) for p in playlists.values], 'int')
plt.hist(playlist_lengths, bins = 50)
plt.axvline(x=20, color="#1DB954")
plt.title('Distribution of number of songs in playlist', size=15);

In [ ]:
# Chart a number of histograms related to track features
start_time = time.time()
fig, ax = plt.subplots(4,3, figsize=(20,10))
columns = ["duration_ms", "count", "danceability", "energy", "tempo", "loudness",
                "speechiness", "acousticness", "instrumentalness", "liveness",
                "artist_popularity", "album_popularity"] #"valence", "key"
col_iter = iter(columns)
for row in range(4):
    for col in range(3):
        sns.distplot(songs[next(col_iter)], ax=ax[row,col])
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Takes about 20 minutes so only uncomment and run intentionally
# Charts pairplot for all features against each other to identify colinear relationships
start_time = time.time()
to_plot = ["duration_ms", "count", "danceability", "energy", "tempo", "loudness",
                "speechiness", "acousticness", "instrumentalness", "liveness",
                "artist_popularity", "album_popularity"] #"valence", "key"
df_to_plot = songs.loc[:,to_plot]
# sns.pairplot(df_to_plot); # Uncomment this to generate pairplot
plt.title("Pairplot for Song Features")
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Takes about 10 minutes so only uncomment and run intentionally
# Distribution plot of loudness vs energy
start_time = time.time()
# sns.jointplot(x="energy", y="loudness", data=songs, kind="kde");
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Takes about 10 minutes so only uncomment and run intentionally
# Distribution plot of danceability vs tempo
start_time = time.time()
# sns.jointplot(x="tempo", y="danceability", data=songs, kind="kde");
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores playlist inclusion by danceability
start_time = time.time()
plt.subplots(figsize=(10,10))
sns.scatterplot(x="count", y="danceability", data=songs)
plt.title("Playlist inclusion rate increases with danceability", size=15)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores playlist inclusion by loudness
start_time = time.time()
plt.subplots(figsize=(10,10))
sns.scatterplot(x="count", y="loudness", data=songs);
plt.title("Loudness is important for inclusion up to a point", size=15)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores playlist inclusion by artist popularity
start_time = time.time()
plt.subplots(figsize=(10,10))
sns.scatterplot(x="count", y="artist_popularity", data=songs);
plt.title("More popular artists are included in more playlists at an increasing rate", size=15)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores playlist inclusion by energy
start_time = time.time()
plt.subplots(figsize=(10,10))
sns.scatterplot(x="count", y="energy", data=songs);
plt.title("Energy has little to no impact on inclusion", size=15)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores playlist inclusion by song duration
start_time = time.time()
plt.subplots(figsize=(10,10))
sns.scatterplot(x="count", y="duration_ms", data=songs);
# plt.axhline(y=350000, color="#1DB954", linestyle='-')
plt.title("Long songs are unlikely to be added to playlists", size=15)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores danceability by tempo
start_time = time.time()
plt.subplots(figsize=(10,10))
sns.scatterplot(x="tempo", y="danceability", data=songs); # The sweet spot of danceability
plt.title("The Tempo Bump: tempos around 125 bpms have much higher danceability", size=15)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores playlist inclusion by album popularity
start_time = time.time()
plt.subplots(figsize=(10,10))
sns.scatterplot(x="count", y="album_popularity", data=songs);
plt.title("Popular songs from unpopular albums (\"One Hit Wonders\") are still included in many playlists ", size=10)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Explores playlist inclusion by tempo excluding significant outliers
# Highlights peak tempo points of notable popularity
plt.subplots(figsize=(20,10))
sns.distplot(songs[(songs.tempo > 30)&(songs.tempo < 230)].tempo, kde=False, bins=200)
plt.axvline(x=80, color="#1DB954", ls="--")
plt.axvline(x=100, color="#1DB954")
plt.axvline(x=120, color="#1DB954")
plt.axvline(x=128, color="#1DB954")
plt.axvline(x=139, color="#1DB954")
plt.axvline(x=170, color="#1DB954")
plt.title("Clear peaks in tempo popularity, suggesting preferred listener tempos", size=20);

In [ ]:
# Derive new release_year column
start_time = time.time()
songs['release_year'] = songs.apply(lambda row: row['album_release_date'][0:4], axis=1)
print(songs[0:5])
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Filter to years with sufficient observations
century_songs = songs[(songs.release_year.astype(int) > 1950) & (songs.release_year.astype(int) < 2017)]

In [ ]:
# Chart album_release_year distribution to understand when songs were made
start_time = time.time()
fig, ax = plt.subplots(figsize=(20,10))
sns.countplot(x="release_year", data=century_songs)
ax.set_xticklabels(ax.get_xticklabels(), rotation=80)
ax.set_title("Album Release Year Distribution on Spotify: 1950 - 2017", size=25)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Scale loudness and tempo to allow plotting on some chart as others
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
century_songs['loudness_scaled'] = sc.fit_transform(np.array(century_songs['loudness']).reshape(-1,1))
display(century_songs['loudness_scaled'][0:5])
century_songs['tempo_scaled'] = sc.fit_transform(np.array(century_songs['tempo']).reshape(-1,1))
display(century_songs['tempo_scaled'][0:5])

In [ ]:
# Chart all track features over time
start_time = time.time()
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(x="release_year", y="danceability", data=century_songs, label="danceability")
sns.lineplot(x="release_year", y="energy", data=century_songs, label="energy")
sns.lineplot(x="release_year", y="liveness", data=century_songs, label="liveness")
sns.lineplot(x="release_year", y="speechiness", data=century_songs, label="speechiness")
sns.lineplot(x="release_year", y="acousticness", data=century_songs, label="acousticness")
sns.lineplot(x="release_year", y="instrumentalness", data=century_songs, label="instrumentalness")
sns.lineplot(x="release_year", y="valence", data=century_songs, label="valence") #, color="#1DB954"
sns.lineplot(x="release_year", y="loudness_scaled", data=century_songs, label="loudness") #, color="#1DB954"
sns.lineplot(x="release_year", y="tempo_scaled", data=century_songs, label="tempo") #, color="#1DB954"
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.set_title("Song Features Change Over Time: 1950 - 2017", size=20);
ax.set_ylabel("Feature level")
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Focus on loudness changes over time
start_time = time.time()
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(x="release_year", y="loudness_scaled", data=century_songs, label="loudness", color="#1DB954")
plt.axvline(x=40, color="red")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45);
ax.set_title("Song Loudness Change Over Time: 1950 - 2017", size=20);
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Focus on danceability changes over time
start_time = time.time()
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(x="release_year", y="danceability", data=century_songs, label="danceability", color="#1DB954")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45);
plt.axvline(x=60, color="red")
ax.set_title("Song Danceability Change Over Time: 1950 - 2017", size=20);
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Investigate musical keys of most popular songs
start_time = time.time()
preferred_artists = songs[songs['count'] > 4050]
print(len(songs), len(preferred_artists))
plt.subplots(figsize=(10,10))
sns.countplot(y="key", data=preferred_artists);
plt.title("Most popular key for Top 100 most included songs", size=15)
print('Time elapsed: {} seconds'.format(round(time.time()-start_time, 3)))

In [ ]:
# Preview of top playlist genres
songs['artist_genres'].value_counts()[0:10]

In [ ]:
# Calculates length of longest and shortest playlists
playlist_lengths = [len(playlist) for playlist in playlists]
print("Largest playlist", max(playlist_lengths))
print("Smallest playlist: ", min(playlist_lengths))